### 0: Libraries

In [9]:
import requests
import pandas as pd
import geopandas as gpd
import json
from datetime import datetime, timedelta
import os
import glob
import warnings
import psycopg2
import sqlalchemy as db
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from shapely.geometry import Point
import contextily as ctx
from geoalchemy2 import Geometry
from pathlib import Path
import ipywidgets as widgets
from ipywidgets import interact

import unittest
from unittest.mock import patch
# warnings
warnings.filterwarnings('ignore')

# Part1: Data Processing

## 1.1 Downloading 311 and Tree data from API
- Download 311 and Tree data using API and Python Code
- 311 Data set is pretty huge, so we choose to download it seperately into subfiles by year, and merge it at the end

In [10]:
#1. Function (1) Manually Doanload Data from NYC Open Data
def download_data(url, app_token, filename, date_field, start_date, end_date, date_format="%Y-%m-%dT%H:%M:%S", limit=10000):
    """
    Downloads data from the specified NYC Open Data URL within a given date range.

    Args:
        url (str): The API endpoint for the dataset.
        app_token (str): Application token for authenticated access.
        filename (str): The name of the file where the data will be saved.
        date_field (str): The name of the date field in the dataset.
        start_date (datetime): The start date for filtering data.
        end_date (datetime): The end date for filtering data.
        date_format (str): Format of the date fields, defaults to '%Y-%m-%dT%H:%M:%S'.
        limit (int): Number of records to retrieve per request, defaults to 10000.

    Returns:
        None: This function writes the downloaded data to a file and does not return anything.
    """
    
    offset = 0
    start_date_str = start_date.strftime(date_format) # Format the start date
    end_date_str = end_date.strftime(date_format) # Format the end date
    # Construct the query for filtering data by date range
    date_query = f"$where={date_field} between '{start_date_str}' and '{end_date_str}'"
    
    first_batch = True  # Flag to identify the first batch of data
    while True:
        # Construct the full URL with necessary query parameters
        full_url = f"{url}?$$app_token={app_token}&{date_query}&$limit={limit}&$offset={offset}"
        response = requests.get(full_url) # Perform the API request

        if response.status_code == 200:
            data = response.text
            records_retrieved = data.count('\n')  # Count the number of lines (records) retrieved

            if first_batch and records_retrieved > 0:  # If this is the first batch and it contains data
                with open(filename, 'w') as file:
                    file.write(data) # Write data to file, including header
                first_batch = False
            elif records_retrieved > 1:  # For subsequent batches, skip the header row
                with open(filename, 'a') as file:
                    file.write(data.split('\n', 1)[1])  # Append data to file without header

            if records_retrieved < limit + 1:   # Check if all records have been retrieved
                break
            offset += limit # Increment the offset for the next batch
        else:
            print(f"Failed to download data at offset {offset}: Status code {response.status_code}")
            break


In [11]:
#app Toekn: Application token used for authentication
app_token = 'Z8lDMDpdnonlT1RjM5YGII6Ii'
#Data URL: Defines the online API URLs for the datasets
url_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv'
url_trees = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.csv'

In [22]:
# download tree data: Initiates the download of tree data
# date format "%m/%d/%Y": Setting the date format to month/day/year
download_data(
    url=url_trees,
    app_token=app_token,  
    filename="data/tree_data.csv",
    date_field="created_at",  
    start_date=datetime(2015, 1, 1),
    end_date=datetime(2015, 12, 31),
    date_format="%m/%d/%Y",  
    limit=10000
)

'# download tree data\n# date format "%m/%d/%Y":\ndownload_data(\n    url=url_trees,\n    app_token=app_token,  # Ensure you use the correct app token here\n    filename="data/tree_data.csv",\n    date_field="created_at",  # Ensure this is the correct field name as per the API documentation\n    start_date=datetime(2015, 1, 1),\n    end_date=datetime(2015, 12, 31),\n    date_format="%m/%d/%Y",  # Adjusted date format to match the dataset\n    limit=10000\n)\n'

In [23]:
# download 311 data from 2015.1.1-2023.9.30
#create a new folder to save 311 data by year
subfolder_name = "311_data"
subfolder_path = os.path.join("data", subfolder_name)
if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)

'# download 311 data from 2015.1.1-2023.9.30\n#create a new folder to save 311 data by year\nsubfolder_name = "311_data"\nsubfolder_path = os.path.join("data", subfolder_name)\nif not os.path.exists(subfolder_path):\n    os.makedirs(subfolder_path)'

In [24]:
# Run download data function by year
#Year 2015
download_data(
    url_311,
    app_token,  
    filename="data/311_data/311_data_2015.csv",
    date_field="created_date",
    start_date=datetime(2015, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM
    end_date=datetime(2015, 12, 31, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=50000
)

'# Run download data function by year\n#Year 2015\ndownload_data(\n    url_311,\n    app_token,  \n    filename="data/311_data/311_data_2015.csv",\n    date_field="created_date",\n    start_date=datetime(2015, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM\n    end_date=datetime(2015, 12, 31, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM\n    date_format="%Y-%m-%dT%H:%M:%S",\n    limit=50000\n)\n\n'

In [12]:
# Unit Test (1)
# Path to the file you expect to exist
filename = "data/311_data/311_data_2015.csv"
# Assert that the file exists
assert os.path.exists(filename), "File does not exist"

In [25]:
# Year 2016
download_data(
    url_311,
    app_token,  
    filename="data/311_data/311_data_2016.csv",
    date_field="created_date",
    start_date=datetime(2016, 1, 1, 0, 0),  
    end_date=datetime(2016, 12, 31, 23, 59, 59),  
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=50000
)

'# Year 2016\ndownload_data(\n    url_311,\n    app_token,  \n    filename="data/311_data/311_data_2016.csv",\n    date_field="created_date",\n    start_date=datetime(2016, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM\n    end_date=datetime(2016, 12, 31, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM\n    date_format="%Y-%m-%dT%H:%M:%S",\n    limit=50000\n)'

In [26]:
# Year 2017
download_data(
    url_311,
    app_token,  
    filename="data/311_data/311_data_2017.csv",
    date_field="created_date",
    start_date=datetime(2017, 1, 1, 0, 0),  
    end_date=datetime(2017, 12, 31, 23, 59, 59),  
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=50000
)

'# Year 2017\ndownload_data(\n    url_311,\n    app_token,  \n    filename="data/311_data/311_data_2017.csv",\n    date_field="created_date",\n    start_date=datetime(2017, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM\n    end_date=datetime(2017, 12, 31, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM\n    date_format="%Y-%m-%dT%H:%M:%S",\n    limit=50000\n)'

In [27]:
# Year 2018
download_data(
    url_311,
    app_token,  
    filename="data/311_data/311_data_2018.csv",
    date_field="created_date",
    start_date=datetime(2018, 1, 1, 0, 0),  
    end_date=datetime(2018, 12, 31, 23, 59, 59),  
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=50000
)

'# Year 2018\ndownload_data(\n    url_311,\n    app_token,  \n    filename="data/311_data/311_data_2018.csv",\n    date_field="created_date",\n    start_date=datetime(2018, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM\n    end_date=datetime(2018, 12, 31, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM\n    date_format="%Y-%m-%dT%H:%M:%S",\n    limit=50000\n)'

In [28]:
# Year 2019
download_data(
    url_311,
    app_token,  
    filename="data/311_data/311_data_2019.csv",
    date_field="created_date",
    start_date=datetime(2019, 1, 1, 0, 0),  
    end_date=datetime(2019, 12, 31, 23, 59, 59),  
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=50000
)

'# Year 2019\ndownload_data(\n    url_311,\n    app_token,  \n    filename="data/311_data/311_data_2019.csv",\n    date_field="created_date",\n    start_date=datetime(2019, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM\n    end_date=datetime(2019, 12, 31, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM\n    date_format="%Y-%m-%dT%H:%M:%S",\n    limit=50000\n)'

In [29]:
# Year 2020
download_data(
    url_311,
    app_token,  
    filename="data/311_data/311_data_2020.csv",
    date_field="created_date",
    start_date=datetime(2020, 1, 1, 0, 0),  
    end_date=datetime(2020, 12, 31, 23, 59, 59),  
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=50000
)

'# Year 2020\ndownload_data(\n    url_311,\n    app_token,  \n    filename="data/311_data/311_data_2020.csv",\n    date_field="created_date",\n    start_date=datetime(2020, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM\n    end_date=datetime(2020, 12, 31, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM\n    date_format="%Y-%m-%dT%H:%M:%S",\n    limit=50000\n)'

In [30]:
# Year 2021
download_data(
    url_311,
    app_token,  
    filename="data/311_data/311_data_2021.csv",
    date_field="created_date",
    start_date=datetime(2021, 1, 1, 0, 0),  
    end_date=datetime(2021, 12, 31, 23, 59, 59),  
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=50000
)

'# Year 2021\ndownload_data(\n    url_311,\n    app_token,  \n    filename="data/311_data/311_data_2021.csv",\n    date_field="created_date",\n    start_date=datetime(2021, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM\n    end_date=datetime(2021, 12, 31, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM\n    date_format="%Y-%m-%dT%H:%M:%S",\n    limit=50000\n)'

In [31]:
# Year 2022
download_data(
    url_311,
    app_token,  
    filename="data/311_data/311_data_2022.csv",
    date_field="created_date",
    start_date=datetime(2022, 1, 1, 0, 0), 
    end_date=datetime(2022, 12, 31, 23, 59, 59),  
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=50000
)

'# Year 2022\ndownload_data(\n    url_311,\n    app_token,  \n    filename="data/311_data/311_data_2022.csv",\n    date_field="created_date",\n    start_date=datetime(2022, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM\n    end_date=datetime(2022, 12, 31, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM\n    date_format="%Y-%m-%dT%H:%M:%S",\n    limit=50000\n)'

In [32]:
# Year 2023
download_data(
    url_311,
    app_token,  
    filename="data/311_data/311_data_2023.csv",
    date_field="created_date",
    start_date=datetime(2023, 1, 1, 0, 0),  
    end_date=datetime(2023, 9, 30, 23, 59, 59),  
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=50000
)

'# Year 2023\ndownload_data(\n    url_311,\n    app_token,  \n    filename="data/311_data/311_data_2023.csv",\n    date_field="created_date",\n    start_date=datetime(2023, 1, 1, 0, 0),  # This represents 2015-01-01 00:00:00 AM\n    end_date=datetime(2023, 9, 30, 23, 59, 59),  # This represents 2023-09-30 11:59:59 PM\n    date_format="%Y-%m-%dT%H:%M:%S",\n    limit=50000\n)'